# **Context Genrator - Usage Example**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Run the following requirements to overcome dill package issues:

In [ ]:
!pip uninstall dill
!pip install dill==0.3.2

In [ ]:
!nvidia-smi

Insert in the variable `your_path` the directory to the Multinews folder in your drive.

In [ ]:
import sys
import os
import pandas as pd
import string
import os

your_path = ""

py_file_location = "/content/drive/MyDrive/" + your_path + "Multinews"
sys.path.append(os.path.abspath(py_file_location))
os.chdir(py_file_location)

In [ ]:
!pip install -q -r './requirements.txt'

In [ ]:
!python -q -m spacy download en_core_web_lg

In [ ]:
import thext_extended
from thext_extended import ContextGenerator

data_path=py_file_location+'/thext_dataset/'
output_dir=py_file_location+'output_context/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_dir=output_dir+'context.csv'

# Dataset name examples
ds_train ='AIPubSumm_train.json'
ds_test ='AIPubSumm_test.json'

cg = ContextGenerator.ContextGenerator()
cg.extract_context(data_path=data_path, ds_train_name=ds_train,ds_test_name=ds_test,output_dir=output_dir)

## **Extracting Hihglights with Generated Contexts**


In [ ]:
from thext_extended import HighlighterContext

path_test_dataset = data_path+ds_test

hc = HighlighterContext.HighlighterContext()
sentences, abstracts = hc.HiglighterGeneration(path_test_dataset, output_dir)

In [ ]:
import HighlighterEvaluation
from thext_extended import Highlighter
from thext_extended import SentenceRankerPlus

base_model_name = "morenolq/thext-ai-scibert"
model_name_or_path = "morenolq/thext-ai-scibert"
sr = SentenceRankerPlus()
sr.load_model(base_model_name=base_model_name, model_name_or_path=model_name_or_path)
h = Highlighter(sr)

list_high=[]
for i in range(len(abstracts)):
  sentence=sentences[i]
  abstract=abstracts[i]
  num_highlights=min(len(sentences[i]),5)
  highlights = h.get_highlights_simple(sentence, abstract,
                NH = num_highlights)
  list_high.append(highlights)

In [ ]:
gold_summaries=[]
df_test = pd.read_json(path_test_dataset)

for i in range(len(df_test['papers'])):

  for k in range(len(df_test['papers'][i]['highlights'])):
    
    string=string+df_test['papers'][i]['highlights'][k]

  gold_summaries.append(string)

In [ ]:
pd.DataFrame(list_high).to_csv(output_dir)

he = HighlighterEvaluation.HighlighterEvaluation()
scores = he.evaluate_highlights_led(list_high, gold_summaries)

print(scores)